In [90]:
# Render our plots inline
%matplotlib inline
%pylab inline  
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# General Plotting Parameters
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['font.size'] = 14.
mpl.rcParams['legend.fontsize'] = 12.
mpl.rcParams['axes.labelsize'] = 12.
mpl.rcParams['xtick.labelsize'] = 10.
mpl.rcParams['ytick.labelsize'] = 10.
mpl.rcParams['xtick.minor.pad'] = 4
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'

#Git says this is patched, but it doesn't work from Pip --upgrade 26-mar-2015
#mpl.rcParams['xtick.minor.visible'] = True  

# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
             (255, 187, 120), (44, 160, 44), (152, 223, 138),
              (148, 103, 189),
             (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127),
             (199, 199, 199), (188, 189, 34), (219, 219, 141),
             (23, 190, 207), (158, 218, 229),(214, 39, 40), (255, 152, 150)]  
    
# Scale the RGB values to the [0, 1] range,
# which is the format matplotlib accepts.  
for i in range(len(tableau20)): 
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [91]:
import calc_enrich
reload(calc_enrich)
from calc_enrich import calc_del_U
from calc_enrich import N_product_by_alpha
from calc_enrich import N_waste_by_alpha
from calc_enrich import machines_per_enr_stage
from calc_enrich import machines_per_strip_stage
from calc_enrich import product_per_enr_stage
from calc_enrich import Npc_from_Nstages
from calc_enrich import Nwc_from_Nstages
from calc_enrich import machines_per_cascade
from calc_enrich import allowed_feed_per_stage
from calc_enrich import calc_feed_flows
from calc_enrich import find_N_stages
from calc_enrich import design_cascade

In [92]:
# centrifuge params 
#omega = 64000
#d = 0.1  # m 
#Z = 2.0   # m
#F_m_hrs = 70 # grams/hr  
#T = 320.0   # K
#cut = 0.5
eff = 1.0 # centrifuge efficiency

d = 0.15  # m 
Z = 1.0   # m
F_m_hrs = 15*60*60/(1e3) # grams/hr  
T = 320.0# K
cut = 0.5
omega = 485/(d/2)

# cascade params
Nfc = 0.007
Npc = 0.035
Nwc = 0.003
Fc_month = 739/10 #kg/month

# The feed enrichment to be used when using a cascade designed for LEU to produce HEU.
Nfc_mod = 0.035
Fc_month_mod = Fc_month #kg/month
#Fc_month_mod = 46 #kg/month

#unit conversions
kg_sec2kg_month = 30.4*24*60*60
v_a = omega * (d/2.0)
F_m = F_m_hrs/(60*60*1000.0)
Fc = Fc_month/kg_sec2kg_month
Fc_mod = Fc_month_mod/kg_sec2kg_month

# Comparison numbers (not needed for calculations)
del_U_th_yr = 1.1 #swu/yr
del_U_obs_yr = 0.71 #Swu/yr
del_U_th = del_U_th_yr/(365.25*24*60*60) #kgSWU/sec
del_U_obs = del_U_obs_yr/(365.25*24*60*60)

Pc_month = 77 #kg/month
Pc = Pc_month/kg_sec2kg_month

qty_len = 6
assay_len = 4

In [98]:
alpha, del_U, del_U_yr = calc_del_U(v_a, Z, d, F_m, T, cut, eff)

# Find the number of stages needed for enriching and stripping 
# (don't use the stages_per_cascade fn because it only works when alpha is close to 1)

ideal_enrich_stage, ideal_strip_stage = find_N_stages(alpha, Nfc, Npc, Nwc)
feed_flows = calc_feed_flows(ideal_enrich_stage, ideal_strip_stage, Fc, cut)
all_stages, n_machines = design_cascade(alpha, del_U, Nfc, Npc, Nwc, feed_flows, verbose=True, qty_len=8)


Stage   #Mach	 Feed    Product  Waste	 F_assay 	P_assay W_assay
0 	21.0 	0.00011117    5.572e-05    5.545e-05    0.007 	0.0094 	0.0052
1 	18.0 	9.264e-05    4.647e-05    4.617e-05    0.0094 	0.0126 	0.007
2 	14.0 	7.411e-05    3.722e-05    3.689e-05    0.0126 	0.017 	0.0094
3 	11.0 	5.559e-05    2.796e-05    2.763e-05    0.017 	0.0227 	0.0126
4 	7.0 	3.706e-05    1.868e-05    1.838e-05    0.0227 	0.0304 	0.017
5 	4.0 	1.853e-05    9.36e-06    9.17e-06    0.0304 	0.0405 	0.0227
-1 	14.0 	7.411e-05    3.712e-05    3.699e-05    0.0052 	0.007 	0.0039
-2 	7.0 	3.706e-05    1.855e-05    1.85e-05    0.0039 	0.0052 	0.0029


In [99]:
for i in range(len(all_stages)):
    if (i == 0):
        p_last = 0
    else:
        p_last = all_stages[i-1][3]
    f_this = all_stages[i][2]
    if (i == len(all_stages)-1):
        w_next = 0
    else:
        w_next = all_stages[i+1][4]
    # If on stage zero then add feed also
    if (i == (ideal_strip_stage)):
        tot_flow = p_last+w_next+Fc
    else:
        tot_flow =  p_last+w_next
    print "stage", i - ideal_strip_stage,"Predicted flow ", feed_flows[i]," actual flow ", tot_flow


stage -2 Predicted flow  3.70573424301e-05  actual flow  3.699e-05
stage -1 Predicted flow  7.41146848603e-05  actual flow  7.4e-05
stage 0 Predicted flow  0.00011117202729  actual flow  0.000111083006823
stage 1 Predicted flow  9.26433560754e-05  actual flow  9.261e-05
stage 2 Predicted flow  7.41146848603e-05  actual flow  7.41e-05
stage 3 Predicted flow  5.55860136452e-05  actual flow  5.56e-05
stage 4 Predicted flow  3.70573424301e-05  actual flow  3.713e-05
stage 5 Predicted flow  1.85286712151e-05  actual flow  1.868e-05


In [95]:
## Check that mass is conserved in the cascade

final_w = all_stages[0][4]
final_p = all_stages[len(all_stages)-1][3]
final_f = round(Fc,qty_len)

res = final_f - final_p - final_w
print "Cascade F\t Cascade P\t Cascade W\t (P+W)/F"
print  final_f, "\t", final_p, "\t", final_w, "\t", (final_p+final_w)/final_f

Cascade F	 Cascade P	 Cascade W	 (P+W)/F
2.8e-05 	0.0 	0.0 	0.0


In [96]:
## Design Cascade based on available number of centrifuges, determine max Fc
max_centrifuges = 100
curr_feed = Fc
optimal_feed = Fc
step_size = 10

ideal_enrich_stage, ideal_strip_stage = find_N_stages(alpha, Nfc, Npc, Nwc)
feed_flows = calc_feed_flows(ideal_enrich_stage, ideal_strip_stage, curr_feed, cut)
all_stages, n_centrifuges = design_cascade(alpha, del_U, Nfc, Npc, Nwc, feed_flows)

optimum_number = False
neg_inc = False
pos_inc = False

if (n_centrifuges < max_centrifuges):
    pos_inc = True
    step = step_size
elif (n_centrifuges > max_centrifuges):
    neg_inc = True
    step = 1/step_size
else:
    optimum_number = True

while (optimum_number == False):
    optimal_feed = curr_feed
    curr_feed *= step
    feed_flows = calc_feed_flows(ideal_enrich_stage, ideal_strip_stage, curr_feed, cut)
    all_stages, n_centrifuges = design_cascade(alpha, del_U, Nfc, Npc, Nwc, feed_flows)
    print all_stages[-1][1]
    if (all_stages[-1][1] < 1):
        optimum_number = True
    elif (neg_inc == True) and (n_centrifuges < max_centrifuges):
        optimum_number = True
    elif (pos_inc == True) and (n_centrifuges > max_centrifuges):
        optimum_number = True

final_feed_flows = calc_feed_flows(ideal_enrich_stage, ideal_strip_stage, optimal_feed, cut)
final_stages, final_n_mach = design_cascade(alpha, del_U, Nfc, Npc, Nwc, final_feed_flows, verbose=True, pretty=True)
print "Cascade can process ", optimal_feed*kg_sec2kg_month, " kg/month"


35.0
Stage   #Mach	 Feed    Product  Waste	 F_assay 	P_assay W_assay
0 	21.0 	292.0    146.36    145.64    0.007 	0.0094 	0.0052
1 	18.0 	243.33    122.06    121.27    0.0094 	0.0126 	0.007
2 	14.0 	194.67    97.76    96.91    0.0126 	0.017 	0.0094
3 	11.0 	146.0    73.43    72.57    0.017 	0.0227 	0.0126
4 	7.0 	97.33    49.05    48.28    0.0227 	0.0304 	0.017
5 	4.0 	48.67    24.59    24.08    0.0304 	0.0405 	0.0227
-1 	14.0 	194.67    97.51    97.16    0.0052 	0.007 	0.0039
-2 	7.0 	97.33    48.73    48.6    0.0039 	0.0052 	0.0029
Cascade can process  73.0  kg/month
